## Figures and Post-Processing Final Analyses for CSU Corpse Flower 

#### Author: Rose K. Rossell, rose.rossell@colostate or rossellrk@gmail.com

In [1]:
# modules to import

import os
import glob
import re

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.cbook as cbook
import matplotlib.colors as colors
import seaborn as sns 

import numpy as np
import math as mat
import scipy.stats as scipy
import statsmodels.api as sm

import matplotlib.dates as md
from matplotlib.dates import DateFormatter
import datetime as datetime
from matplotlib.ticker import FormatStrFormatter
from matplotlib.ticker import ScalarFormatter

import time as time

from collections import defaultdict

import molmass
from molmass import Formula

In [2]:
# todays date for saving purposes 
today = datetime.date.today().strftime('%Y%m%d')

In [3]:
# setting default font as Arial for everything 
plt.rc('font',family = 'Arial')

### Defining functions to use during the analysis

In [5]:
# function to remove quotes from compound names in column headers
def remove_quotes(text):
    chars_to_replace = "'" # want to remove single quotes
    for char in chars_to_replace:
       text = text.replace(char, "") # replace the defined character with nothing
    return text

# function to read chemical formulas
def parse_formula(chem_formula): # create a function that counts the number of elements in the compound names
    pattern = r'([A-Z][a-z]?)(\d*)' # pattern: Big Letter with potenital second small letter (covers C and He, for example), followed by a # (default 1)
    counts = defaultdict(int) # creates a dictionary of the counted values
    for (element, count) in re.findall(pattern, chem_formula): # recalls the set pattern and then perfroms the count for each compound
        counts[element] += int(count) if count else 1 # starts with a default count of 1 for each element
    return counts

# function to remove the extra H+ from the PTR formulas
def remove_PTR_Hion(list_of_formulas):
    chem_formulas = []   # create an empty list
    for x in list_of_formulas: # for every value in the list:
        formula = parse_formula(x) # apply the chemical formula to each compound 
        num_h = formula.get('H', 0) # get the number of atoms for the 4 elements
        true_h = num_h - 1    # subtract off one H
        num_o = formula.get('O', 0) 
        num_c = formula.get('C', 0)
        num_s = formula.get('S', 0)
        if num_c > 0: # write the values in the proper form for each element
            if num_c == 1:
                carbon = 'C'
            else:
                carbon = 'C' + str(num_c) # convert the numeric value to string format
        else:
            carbon = str('')
        if num_o > 0:
            if num_o == 1:
                oxygen = 'O'
            else:
                oxygen = 'O' + str(num_o)
        else:
            oxygen = str('')
        if num_s > 0:
            if num_s == 1:
                sulfur = 'S'
            else:
                sulfur = 'S' + str(num_s)
        else:
            sulfur = str('')
        if true_h > 0:
            if true_h == 1:
                hydrogen = 'H'
            else:
                hydrogen = 'H' + str(true_h)
        else:
            hydrogen = str('')
        new_formula = carbon + hydrogen + oxygen + sulfur # combine the values together
        chem_formulas.append((new_formula)) # store in the list
    return chem_formulas

# function to calculate elemental ratios for Van Krevelan plots
def calc_ratios(compounds_list): 
    ratios = [] # create empty list
    for x in compounds_list: # for every cmpd in the cmpd list:
        counts = parse_formula(x) # perform the formula function, defined in the first code block when reading in the PTR file
        num_o = counts.get('O', 0) # calc the ratios based on the values stored in the created dictionary for each cmpd
        num_h = counts.get('H', 0)
        num_c = counts.get('C', 0)
        num_s = counts.get('S', 0)
        if num_c > 0: # calc the ratio if the # of C >1
            ratio_h_c = num_h / num_c
            ratio_s_c = num_s / num_c
            ratio_o_c = num_o / num_c
        else:
            ratio_h_c = float('inf') # otherwise just store an inf value
            ratio_s_c = float('inf')
            ratio_o_c = float('inf')
        ratios.append((x, ratio_h_c, ratio_s_c, ratio_o_c)) # add the ratios to the empty list we created earlier with the compound names (x variable)
    df = pd.DataFrame(ratios, columns = ['compound', 'H:C', 'S:C', 'O:C']) # create a df from the stored list 
    df['H:C'] = df['H:C'].round(2) # round the ratios to 2 decimal places
    df['S:C'] = df['S:C'].round(2)
    df['O:C'] = df['O:C'].round(2)
    return df # return the new df

# function to calculate number densities from ppb space
avo_constant = 6.022E23
R_constant = 0.08206
def number_density(conc_ppb, temp_c, atm_pr_hpa):
    temp_k = temp_c + 273.13 # conver to K
    pressure_atm = atm_pr_hpa * 0.000987 # convert from hPa to atm
    vol_conv = 1000 # L to mL to cm3
    
    number_density_value = ((conc_ppb * avo_constant * 1E-9) / ((R_constant * temp_k) / pressure_atm)) / vol_conv 
    return(number_density_value)

# function to average data to the GC sampling time blocks
def calc_time_block_stats(df, name): # stats to apply to each df
    stat_funcs = [df.mean, df.std, df.max, df.min, df.median]
    stat_names = ['mean', 'stdev', 'max', 'min', 'median']
    stats = pd.concat( # combining the stats for each sample block into a df and renaming each stat with the correct name
        [pd.DataFrame(func()).transpose().rename(index = {0 : name}) for func, name in zip(stat_funcs, stat_names)] 
    )
    stats.drop(columns = 'datetime', inplace = True) # dropping the datetime column b/c they are sorted by datatime
    stats.index = pd.MultiIndex.from_product([[name], stats.index], names = ['GC_Sample', 'stats']) # adding a multilevel index w/ the df name
    return stats

# function to add subscripts to compound nnames
def add_subscripts(list_df_headers):
    new_col_headers = []   # create an empty list
    for x in list_df_headers: # for every value in the list
        formula = parse_formula(x)
        num_h = formula.get('H', 0) # get the number of atoms for the 4 elements
        num_o = formula.get('O', 0) 
        num_c = formula.get('C', 0)
        num_s = formula.get('S', 0)
        if num_c > 0:
            if num_c == 1:
                carbon = 'C' # if just one atom just write the letter
            else: 
                carbon = f"$C_{{{num_c}}}$" # otherwise embed the value as a subscript next to the letter using f-strings notation
        else:
            carbon = str('')
        if num_h > 0:
            if num_h == 1:
                hydrogen = 'H'
            else:
                hydrogen = f"$H_{{{num_h}}}$"
        else:
            hydrogen = str('')
        if num_o > 0:
            if num_o == 1:
                oxygen = 'O'
            else:
                oxygen = f"$O_{{{num_o}}}$"
        else:
            oxygen = str('')
        if num_s > 0:
            if num_s == 1:
                sulfur = 'S'
            else:
                sulfur = f"$S_{{{num_s}}}$"
        else:
            sulfur = str('')
        new_formula = carbon + hydrogen + oxygen + sulfur + f"$^+$" # combine the values together and add a superscripted + b/c of PTR
        new_col_headers.append((new_formula)) # store in the list
    return new_col_headers # return list of updated names

### Pre-Plot Code to Run

In [6]:
# list of suspected fragment ions (or cmpds without spikes during bloom period) to drop from analyses: 

suspect_frag_ions = ['CHS+', 'CH3S+', 'CH3S2+', 'CH5O3+', 'CH5S2+', 'CH6N2OS+', 'C2H5S+', 'C2H5S2+', 
                     'C2H6S2+', 'C7H11O+', 'S2+', 'CH3O2+', 'C7H7O2+', 'C9H11O2+', 'C9H11O3+', 'CH7OS+'] # edit as needed
suspect_frag_molcs = remove_PTR_Hion(suspect_frag_ions) # apply function to get actual molecule

#### Reading in the PTR datafile

In [ ]:
import datetime as datetime # keep getting a weird error and having to reimport this module
 
corpse_ts_file = 'PTR_ppb_All_Cmpds_20241106.csv' # edit as needed # PTR file is saved in the same folder as the code file
corpse_ts = pd.read_csv(corpse_ts_file)


corpse_ts['date'] = pd.to_datetime(corpse_ts['date']) # turn date col into pd.DateTime
corpse_ts['time'] = pd.to_datetime(corpse_ts['time'], format = '%H:%M:%S') # turn time col into pd.DateTime
corpse_ts['time'] = corpse_ts['time'].dt.time # drop weird date format from the time col 

# combine into new datetime col
corpse_ts['datetime'] = corpse_ts.apply(lambda row: datetime.datetime.combine(row['date'], row['time']), axis = 1) 
datetime = corpse_ts['datetime']
corpse_ts.drop(columns = ['LocalTime', 'date', 'time'], inplace = True) # drop old date and time columns

datetime = corpse_ts['datetime']# store datetime for future

# apply remove_quotes function to columns in the corpse df
corpse_ts.columns = [remove_quotes(col) for col in corpse_ts.columns ] 

# re-arranging df so that all S-spp are together
corpse_ts.rename(columns = {'CH5S_actual_ppb' : 'CH5S+'}, inplace = True)
CH5S = corpse_ts.pop('CH5S+')
corpse_ts.insert(2, 'CH5S+', CH5S)
S2 = corpse_ts.pop('S2+')
corpse_ts.insert(2, 'S2+', S2)
C2H7O2S = corpse_ts.pop('C2H7O2S+')
corpse_ts.insert(2, 'C2H7O2S+', C2H7O2S)
C4H9OS = corpse_ts.pop('C4H9OS+')
corpse_ts.insert(2, 'C4H9OS+', C4H9OS)

corpse_ts_PTR_ions = corpse_ts.copy() # saving version containing the PTR ions just in case

# create df with the actual cmpd formula (no PTR H+)
corpse_ts_actual_cmpds = corpse_ts.copy() # no overwriting !
# drop the date time column so it doesn't go through the column function
corpse_ts_actual_cmpds.drop(columns = ['datetime'], inplace = True) 

compound_list = list(corpse_ts_actual_cmpds.columns) # get a list of the compound names (aka each column header)

# apply the function to remove the PTR H+ from each compound name
formula_names = remove_PTR_Hion(compound_list) 

corpse_ts_actual_cmpds.columns = formula_names # rename all the columnds with the new, whole molecule names
corpse_ts_actual_cmpds['datetime'] = datetime # put the datetime column back in the df 

#print(corpse_ts)

#### Reading in the metadata files

In [7]:
co2_file = 'corpse_CO2_monitor_data.csv' # datafile is stored in the same folder as the source code
co2_ts = pd.read_csv(co2_file)
co2_ts['datetime'] = pd.to_datetime(co2_ts['Time(DD/MM/YYYY H:mm:ss)'], dayfirst = True) # formating the datetime column correctly as a datetime type
co2_ts.drop(columns = ['Time(DD/MM/YYYY H:mm:ss)'], inplace = True) # removing original one

# reading in the preliminary lifetime and rate constant datafile: 20241017
lifetime_file = 'prelim_lifetimes_rate_constants.csv' # datafile is stored in the same folder as the source code
lifetimes_RateConstants = pd.read_csv(lifetime_file)

lifetimes_RateConstants.reset_index(inplace = True)
lifetimes_RateConstants.drop(columns = ['index'], inplace = True) # resetting the index is adding an index colum to df that needs to be dropped


# reading in the GC data
gc_avg_ppb_file = 'gc_cmpds_avg_ppb.csv' # datafile is stored in the same folder as the source code
gc_avg_ppb = pd.read_csv(gc_avg_ppb_file)
gc_avg_ppb['start_datetime'] = gc_avg_ppb['date'] + ' ' + gc_avg_ppb['Start_Time'] # combining into a datetime column
gc_avg_ppb['start_datetime'] = pd.to_datetime(gc_avg_ppb['start_datetime']) # converting to pd datetime class
gc_avg_ppb['end_datetime'] = gc_avg_ppb['date'] + ' ' + gc_avg_ppb['End_Time']
gc_avg_ppb['end_datetime'] = pd.to_datetime(gc_avg_ppb['end_datetime']) 
gc_avg_ppb.drop(columns = ['date', 'Start_Time', 'End_Time'], inplace = True) # dropping for clarity
 


### Analysis code chunks

In [9]:
# seperating the female and male time blocks

female_ts = corpse_ts_PTR_ions.copy() # keeping the PTR formula for the corr matrices
female_ts.set_index('datetime', inplace = True) # set datetime as index to easily splice dataframe
female_ts = female_ts['2024-05-25 18:00:00' : '2024-05-26 9:00:00'] # Female  bloom = 6pm 20240525 to 9am 20240526
female_ts.reset_index(inplace = True) # reset index 

male_ts = corpse_ts_PTR_ions.copy()
male_ts.set_index('datetime', inplace = True) 
male_ts = male_ts['2024-05-26 18:00:00' : '2024-05-27 9:00:00'] # Male  bloom = 6pm 20240526 to 9am 20240527
male_ts.reset_index(inplace = True)

NameError: name 'corpse_ts_PTR_ions' is not defined

In [ ]:
# summing the totals of each compound for the male and female emission periods (all PTR traces)

total_female_emissions = female_ts.copy() # no overwriting !
total_female_emissions.drop(columns = ['datetime'], inplace = True) # don't need datetime
total_female_emissions = pd.DataFrame(total_female_emissions.sum()) # add every row per column
total_female_emissions.rename(columns = {0 : 'total_compound_amt_ppb_female'}, inplace = True) # rename the column
total_female_emissions = pd.DataFrame(total_female_emissions) # dataframe it up

total_male_emissions = male_ts.copy() # rinse and repeat
total_male_emissions.drop(columns = ['datetime'], inplace = True)
total_male_emissions = pd.DataFrame(total_male_emissions.sum())
total_male_emissions.rename(columns = {0 : 'total_compound_amt_ppb_male'}, inplace = True)
total_male_emissions = pd.DataFrame(total_male_emissions)

male_vs_female_totals = pd.concat([total_female_emissions, total_male_emissions], axis = 1).reindex(total_female_emissions.index) # concat the df 
male_vs_female_totals.index.name = 'compound' # index by compound name to make future concats easier

male_vs_female_totals = male_vs_female_totals.transpose() # rotate that bitch so male and female are the row headings - makes bar graphs easier



In [ ]:
# importing meta data for cmpds we have info on

ordered_vp = lifetimes_RateConstants.copy() # no over-writing !
ordered_vp = ordered_vp.sort_values(by = 'vapor_pressure_mmHg', ascending = True) # sorting by vapor pressure
ordered_vp.rename(columns = {'cmpd_PTR_formula' : 'compound'}, inplace = True) # re-name column for future concatting
cmpds_with_info = ordered_vp['compound'] # get list of compound names to splice the male and female totals df
ordered_vp.set_index('compound', inplace = True) # set as index for future concatting

# need male and female totals for only the compounds we have data from

copy = male_vs_female_totals.copy() # no over-writing !
vp_cmpds_male_female = copy[cmpds_with_info] # splicing to only include compounds that we have data for
vp_cmpds_male_female = vp_cmpds_male_female.transpose() # flip df so compounds are now the index for future concatting
female_totals = vp_cmpds_male_female['total_compound_amt_ppb_female'].sum() # sum of all of the compounds emitted during the female bloom
male_totals = vp_cmpds_male_female['total_compound_amt_ppb_male'].sum() # sum of all of the compounds emitted during the male bloom
vp_cmpds_male_female['cmpd_percent_female'] =  vp_cmpds_male_female['total_compound_amt_ppb_female'] / female_totals * 100 # might want percentage 
vp_cmpds_male_female['cmpd_percent_male'] =  vp_cmpds_male_female['total_compound_amt_ppb_male'] / male_totals * 100

vp_react_emission_concat = pd.concat([ordered_vp, vp_cmpds_male_female], axis = 1)
vp_react_emission_concat['k_o3_298K'] = vp_react_emission_concat['k_o3_298K'].fillna(0)


In [ ]:
# repeating from above

cmpd_names = vp_react_emission_concat.index # need list of cmpds that we have data for
cmpds = corpse_ts_PTR_ions.copy() # no over writing !
co2_copy = co2_ts.copy()

# splicing the CO2 data into male and female bloom times 

co2_female = co2_copy.copy()
co2_female.set_index('datetime', inplace = True) # setting index to dt class for splicing 
co2_female = co2_female.loc['2024-05-25 18:01' : '2024-05-26 9:00'] # splice dataset to female times (6pm to 9am)
co2_female.reset_index(inplace = True) 
co2_female.rename(columns = {'datetime' : 'datetime_co2'}, inplace = True) # just in case we get confused later
co2_female['datetime_co2'] = co2_female['datetime_co2'].dt.strftime('%Y-%m-%d %H:%M') # re-formatting the time 

co2_male = co2_copy.copy() # rinse and repeat
co2_male.set_index('datetime', inplace = True)
co2_male.index = pd.to_datetime(co2_male.index)
co2_male = co2_male.loc['2024-05-25 18:01' : '2024-05-26 9:00']
co2_male.reset_index(inplace = True)
co2_male.rename(columns = {'datetime' : 'datetime_co2'}, inplace = True)
co2_male['datetime_co2'] = co2_male['datetime_co2'].dt.strftime('%Y-%m-%d %H:%M')

# need to re-sample the emission data by 2 min to match the CO2 data

female_2min_avg = cmpds.copy()
female_2min_avg.set_index('datetime', inplace = True) # setting index to dt class for splicing 
female_2min_avg = female_2min_avg['2024-05-25 18:01:00' : '2024-05-26 9:00:00'] # Female  bloom = 6pm to 9am 
female_2min_avg = female_2min_avg.resample('2min', offset = '1min').mean() 
female_2min_avg.reset_index(inplace = True) # resampling sets the datetime col to the index, so need to re-index
female_2min_avg['datetime'] = female_2min_avg['datetime'].dt.strftime('%Y-%d-%m %H:%M') # round to every minute
female_2min_avg.rename(columns = {'datetime' : 'datetime_cmpds'}, inplace = True)
#print(female_2min_avg)


male_2min_avg = cmpds.copy()
male_2min_avg.set_index('datetime', inplace = True) # setting index to dt class for splicing 
male_2min_avg = male_2min_avg['2024-05-26 18:01:00' : '2024-05-27 9:00:00'] # Male  bloom = 6pm to 9am 
male_2min_avg = male_2min_avg.resample('2min', offset = '1min').mean() 
male_2min_avg.reset_index(inplace = True) # resampling sets the datetime col to the index, so need to re-index
male_2min_avg['datetime'] = male_2min_avg['datetime'].dt.strftime('%Y-%d-%m %H:%M') # round to every minute
male_2min_avg.rename(columns = {'datetime' : 'datetime_cmpds'}, inplace = True)
#print(male_2min_avg)


# converting from ppb to number density 

female_num_den = female_2min_avg.copy() 
for index, row in female_num_den.iterrows(): # applying number density function defined above, using data from the correct CO2 df
    temp_c = co2_female.loc[index, 'Temperature(°C)']
    atm_pr_hpa = co2_female.loc[index, 'Atmospheric pressure(hPa)']

    for col in row.index:
         if pd.api.types.is_numeric_dtype(female_num_den[col]):
             female_num_den.at[index, col] = number_density(
                 row[col],
                 temp_c,
                 atm_pr_hpa
             )
female_number_den = female_num_den.copy()

male_num_den = male_2min_avg.copy()
for index, row in male_num_den.iterrows(): # applying number density function defined above, using data from the correct CO2 df
    temp_c = co2_male.loc[index, 'Temperature(°C)']
    atm_pr_hpa = co2_male.loc[index, 'Atmospheric pressure(hPa)']

    for col in row.index:
         if pd.api.types.is_numeric_dtype(male_num_den[col]):
             male_num_den.at[index, col] = number_density(
                 row[col],
                 temp_c,
                 atm_pr_hpa
             )
male_number_den = male_num_den.copy()



In [ ]:
# only calculating reactivites at midnight

# first, need to extract the row that contains only the data for midnight
copy = female_num_den.copy()
female_midnight = copy.loc[(copy['datetime_cmpds'] >= '2024-25-05 23:55')
    & (copy['datetime_cmpds'] < '2024-26-05 00:05')]
female_midnight = female_midnight.loc[[179]] # selecting index # 179 b/c that is closest to midnight
female_midnight.drop(columns = ['datetime_cmpds'], inplace = True) # dropping datetime col 
female_midnight_reactivies = female_midnight[cmpd_names] # only pulling out compounds we have data for
female_midnight_reactivies = female_midnight_reactivies.transpose() # want cmpds as row labels
female_midnight_reactivies.rename(columns = {179 : 'female_num_den_midnight'}, inplace = True) # renaming num den column
female_midnight_reactivies['no3_reactivity_1/s'] = vp_react_emission_concat['k_no3_298K'] * female_midnight_reactivies['female_num_den_midnight'] 
female_midnight_reactivies['o3_reactivity_1/s'] = vp_react_emission_concat['k_o3_298K'] * female_midnight_reactivies['female_num_den_midnight']
female_midnight_reactivies['oh_reactivity_1/s'] = vp_react_emission_concat['k_OH_298K'] * female_midnight_reactivies['female_num_den_midnight']
female_midnight_reactivies['proposed_cmpd_name'] = vp_react_emission_concat['proposed_cmpd_name'] # want the actual names to come too
# contribution of each compound to the total reactivity

female_midnight_reactivies['no3_reactivity_percent'] = female_midnight_reactivies['no3_reactivity_1/s'] / (female_midnight_reactivies['no3_reactivity_1/s'].sum()) * 100
female_midnight_reactivies['o3_reactivity_percent'] = female_midnight_reactivies['o3_reactivity_1/s'] / (female_midnight_reactivies['o3_reactivity_1/s'].sum()) * 100
female_midnight_reactivies['oh_reactivity_percent'] = female_midnight_reactivies['oh_reactivity_1/s'] / (female_midnight_reactivies['oh_reactivity_1/s'].sum()) * 100
female_midnight_reactivies.reset_index(inplace = True)
female_midnight_reactivies.rename(columns = {'index' : 'compounds'}, inplace = True)


copy = male_num_den.copy()
male_midnight = copy.loc[(copy['datetime_cmpds'] >= '2024-26-05 23:55')
    & (copy['datetime_cmpds'] < '2024-27-05 00:05')]
male_midnight = male_midnight.loc[[179]] # selecting index # 179 b/c that is closest to midnight
male_midnight.drop(columns = ['datetime_cmpds'], inplace = True)
male_midnight_reactivies = male_midnight[cmpd_names] # only pulling out compounds we have data for
male_midnight_reactivies = male_midnight_reactivies.transpose() # want cmpds as row labels
male_midnight_reactivies.rename(columns = {179 : 'male_num_den_midnight'}, inplace = True) # renaming num den column
male_midnight_reactivies['no3_reactivity_1/s'] = vp_react_emission_concat['k_no3_298K'] * male_midnight_reactivies['male_num_den_midnight'] 
male_midnight_reactivies['o3_reactivity_1/s'] = vp_react_emission_concat['k_o3_298K'] * male_midnight_reactivies['male_num_den_midnight']
male_midnight_reactivies['oh_reactivity_1/s'] = vp_react_emission_concat['k_OH_298K'] * male_midnight_reactivies['male_num_den_midnight']
male_midnight_reactivies['proposed_cmpd_name'] = vp_react_emission_concat['proposed_cmpd_name'] # want the actual names to come too
# contribution of each compound to the total reactivity

male_midnight_reactivies['no3_reactivity_percent'] = male_midnight_reactivies['no3_reactivity_1/s'] / (male_midnight_reactivies['no3_reactivity_1/s'].sum()) * 100
male_midnight_reactivies['o3_reactivity_percent'] = male_midnight_reactivies['o3_reactivity_1/s'] / (male_midnight_reactivies['o3_reactivity_1/s'].sum()) * 100
male_midnight_reactivies['oh_reactivity_percent'] = male_midnight_reactivies['oh_reactivity_1/s'] / (male_midnight_reactivies['oh_reactivity_1/s'].sum()) * 100
male_midnight_reactivies.reset_index(inplace = True)
male_midnight_reactivies.rename(columns = {'index' : 'compounds'}, inplace = True)



In [ ]:
# what if we only want the top 3 reactive compounds?

# top 3 NO3 reactive cmpds (female)
top_female_no3_reactivies_1 = female_midnight_reactivies.sort_values('no3_reactivity_percent', ascending = False)
top_female_reactivies = top_female_no3_reactivies_1[:2].copy() # selecting top 2
others_df = pd.DataFrame(data = {
    'compounds' : ['Other'], # naming the other cmpds
    'proposed_cmpd_name' : ['Other'],
    'female_num_den_midnight' : [top_female_no3_reactivies_1['female_num_den_midnight'][2:].sum()], # adding up the remaining cmpds (not top 3)
    'no3_reactivity_1/s' : [top_female_no3_reactivies_1['no3_reactivity_1/s'][2:].sum()],
    'no3_reactivity_percent' : [top_female_no3_reactivies_1['no3_reactivity_percent'][2:].sum()],
    'o3_reactivity_1/s' : [top_female_no3_reactivies_1['o3_reactivity_1/s'][2:].sum()], # not relevant but need to include b/c they r in the other df
    'o3_reactivity_percent' : [top_female_no3_reactivies_1['o3_reactivity_percent'][2:].sum()],
     })
top_female_no3_reactivies = pd.concat([top_female_reactivies, others_df])


# top 3 O3 reactive cmpds (female)
top_female_o3_reactivies_1 = female_midnight_reactivies.sort_values('o3_reactivity_percent', ascending = False)
top_female_reactivies = top_female_o3_reactivies_1[:2].copy() # selecting top 2
others_df = pd.DataFrame(data = {
    'compounds' : ['Other'],
    'proposed_cmpd_name' : ['Other'],
    'female_num_den_midnight' : [top_female_o3_reactivies_1['female_num_den_midnight'][2:].sum()], # adding up the remaining cmpds (not top 3)
    'no3_reactivity_1/s' : [top_female_o3_reactivies_1['no3_reactivity_1/s'][2:].sum()], # not relevant but need to include b/c they r in the other df
    'no3_reactivity_percent' : [top_female_o3_reactivies_1['no3_reactivity_percent'][2:].sum()],
    'o3_reactivity_1/s' : [top_female_o3_reactivies_1['o3_reactivity_1/s'][2:].sum()],
    'o3_reactivity_percent' : [top_female_o3_reactivies_1['o3_reactivity_percent'][2:].sum()],
    })
top_female_o3_reactivies = pd.concat([top_female_reactivies, others_df])
top_female_o3_reactivies[top_female_o3_reactivies['o3_reactivity_percent'] < 0] = False # getting -% so setting negative values to False for graphing



# top 3 NO3 reactive cmpds (male)
top_male_no3_reactivies_1 = male_midnight_reactivies.sort_values('no3_reactivity_percent', ascending = False)
top_male_reactivies = top_male_no3_reactivies_1[:2].copy()
others_df = pd.DataFrame(data = {
    'compounds' : ['Other'],
    'proposed_cmpd_name' : ['Other'],
    'male_num_den_midnight' : [top_male_no3_reactivies_1['male_num_den_midnight'][2:].sum()],
    'no3_reactivity_1/s' : [top_male_no3_reactivies_1['no3_reactivity_1/s'][2:].sum()],
    'no3_reactivity_percent' : [top_male_no3_reactivies_1['no3_reactivity_percent'][2:].sum()],
    'o3_reactivity_1/s' : [top_male_no3_reactivies_1['o3_reactivity_1/s'][2:].sum()],
    'o3_reactivity_percent' : [top_male_no3_reactivies_1['o3_reactivity_percent'][2:].sum()],
    })
top_male_no3_reactivies = pd.concat([top_male_reactivies, others_df])


# top 3 O3 reactive cmpds (male)
top_male_o3_reactivies_1 = male_midnight_reactivies.sort_values('o3_reactivity_percent', ascending = False)
top_male_reactivies = top_male_o3_reactivies_1[:2].copy()
others_df = pd.DataFrame(data = {
    'compounds' : ['Other'],
    'proposed_cmpd_name' : ['Other'],
    'male_num_den_midnight' : [top_male_o3_reactivies_1['male_num_den_midnight'][2:].sum()],
    'no3_reactivity_1/s' : [top_male_o3_reactivies_1['no3_reactivity_1/s'][2:].sum()],
    'no3_reactivity_percent' : [top_male_o3_reactivies_1['no3_reactivity_percent'][2:].sum()],
    'o3_reactivity_1/s' : [top_male_o3_reactivies_1['o3_reactivity_1/s'][2:].sum()],
    'o3_reactivity_percent' : [top_male_o3_reactivies_1['o3_reactivity_percent'][2:].sum()],
    })
top_male_o3_reactivies = pd.concat([top_male_reactivies, others_df])



# top 3 OH reactive cmpds (female)
top_female_oh_reactivies_1 = female_midnight_reactivies.sort_values('oh_reactivity_percent', ascending = False)
top_female_reactivies = top_female_oh_reactivies_1[:2].copy()
others_df = pd.DataFrame(data = {
    'compounds' : ['Other'],
    'proposed_cmpd_name' : ['Other'],
    'female_num_den_midnight' : [top_female_oh_reactivies_1['female_num_den_midnight'][2:].sum()],
    'no3_reactivity_1/s' : [top_female_oh_reactivies_1['no3_reactivity_1/s'][2:].sum()],
    'no3_reactivity_percent' : [top_female_oh_reactivies_1['no3_reactivity_percent'][2:].sum()],
    'o3_reactivity_1/s' : [top_female_oh_reactivies_1['o3_reactivity_1/s'][2:].sum()],
    'o3_reactivity_percent' : [top_female_oh_reactivies_1['o3_reactivity_percent'][2:].sum()],
    'oh_reactivity_1/s' : [top_female_oh_reactivies_1['oh_reactivity_1/s'][2:].sum()],
    'oh_reactivity_percent' : [top_female_oh_reactivies_1['oh_reactivity_percent'][2:].sum()],
    })
top_female_oh_reactivies = pd.concat([top_female_reactivies, others_df])


# top 3 OH reactive cmpds (male)
top_male_oh_reactivies_1 = male_midnight_reactivies.sort_values('oh_reactivity_percent', ascending = False)
top_male_reactivies = top_male_oh_reactivies_1[:2].copy()
others_df = pd.DataFrame(data = {
    'compounds' : ['Other'],
    'proposed_cmpd_name' : ['Other'],
    'male_num_den_midnight' : [top_male_oh_reactivies_1['male_num_den_midnight'][2:].sum()],
    'no3_reactivity_1/s' : [top_male_oh_reactivies_1['no3_reactivity_1/s'][2:].sum()],
    'no3_reactivity_percent' : [top_male_oh_reactivies_1['no3_reactivity_percent'][2:].sum()],
    'o3_reactivity_1/s' : [top_male_oh_reactivies_1['o3_reactivity_1/s'][2:].sum()],
    'o3_reactivity_percent' : [top_male_oh_reactivies_1['o3_reactivity_percent'][2:].sum()],
    'oh_reactivity_1/s' : [top_male_oh_reactivies_1['oh_reactivity_1/s'][2:].sum()],
    'oh_reactivity_percent' : [top_male_oh_reactivies_1['oh_reactivity_percent'][2:].sum()],
    })
top_male_oh_reactivies = pd.concat([top_male_reactivies, others_df])


In [ ]:
# Calculating emission rates uing average compound concentations

greenhouse_vol = 2870 * 12 # value in ft^3
greenhouse_vol = greenhouse_vol / 35.31468482  # converting ft^3 to m^3
greenhouse_exch_rate = 81 # units = 1/hour # average of front and back avg rates from CO2 experiment on 20240907

# finding the avg number density for each compound over each bloom stage:
copy = female_number_den.copy() # no overwriting !
copy.drop(columns = 'datetime_cmpds', inplace = True) # get rid of dt column - can't average it 
data = copy.mean() # take the mean of each column/compound
female_avg_num_den = pd.DataFrame(data = data, columns = ['female_avg_num_den']) # made df
female_avg_num_den.index.name = 'compounds' # name index
copy = male_number_den.copy() # no overwriting !
copy.drop(columns = 'datetime_cmpds', inplace = True) # get rid of dt column - can't average it 
data = copy.mean() # take the mean of each column/compound
male_avg_num_den = pd.DataFrame(data = data, columns = ['male_avg_num_den']) # made df
male_avg_num_den.index.name = 'compounds' # name index

avg_num_den = pd.concat([female_avg_num_den, male_avg_num_den], axis = 1) # combining into 1 df along shared index

# need list of compound names to remove H+ from name
ignore = 'datetime_cmpds'
compound_list = avg_num_den.index # cmpds are index of this df
for col in compound_list:
    if col != ignore:
        formula_names = remove_PTR_Hion(compound_list) # apply forumla to remove additional H+ from PTR formulas

# need molecular weights for each cmpd to calc emission rates
molecular_weights = [] # creating empty list
for x in formula_names: 
    mw = Formula(x).mass # applying Formula formula from MolarMass library
    molecular_weights.append(mw) # creating new list of MW's 


# converting molc/cm3 into g/m3
g_m3_avgs = avg_num_den / avo_constant # converting from molc/cm3 to mole/cm3 
g_m3_avgs = g_m3_avgs.mul(molecular_weights, axis = 'index') # converting from mole to g
g_m3_avgs = g_m3_avgs.mul(1E6, axis = 'index') # converting from mL to m3 

ER_avgs_g_hour = g_m3_avgs * greenhouse_exch_rate * greenhouse_vol

ER_no_frags_g_hour = ER_avgs_g_hour.transpose() # want cmpds as column names to remove fragments
ER_no_frags_g_hour.drop(columns = suspect_frag_ions, inplace = True) # removing suspected fragments 
ER_no_frags_g_hour = ER_no_frags_g_hour.transpose()
ER_no_frags_g_hour.rename(columns = {'female_avg_num_den' : 'female_g_hour', 'male_avg_num_den' : 'male_g_hour'}, inplace = True)




In [ ]:
no_frags_MvF = male_vs_female_totals.copy()
no_frags_MvF.drop(columns = suspect_frag_ions, inplace = True)
totals = no_frags_MvF.sum(axis = 1) # add the total amount for both male and female
totals_ppm = totals / 1000 # converting to ppm to get rid of some trailing zeros
percentage = no_frags_MvF.div(totals, axis = 0).mul(100).round(2) # get percents of total

graphing_emiss_rates = ER_no_frags_g_hour.transpose() # need cmpds as col names for graphing bar charts

col_headers = graphing_emiss_rates.columns # getting list of column labels (compound names)
formatted_cmpd_names = add_subscripts(col_headers) # applying subscript formatter function

In [ ]:
# exposure calcs

# [VOC reacted away] = [Initial VOC] (molc/cm3) * kvoc-no3 (cm3 / molc s) * [no3] (ppt) * exposure time (s)
# [VOC remaining] = [Initial VOC] - [VOC reacted away]

pristine_no3_conc_ppb = 2 / 1000
polluted_no3_conc_ppb = 15 / 1000
exposure_time_sec = 3600

# setting up the inital conc df
cmpd_k_no3 = vp_react_emission_concat[['k_no3_298K']].copy() # need all legends to pull from data that won't change based on exposure
copy = male_ts.copy()
male_midnight = copy.loc[(copy['datetime'] >= '2024-05-26 23:55') # using midnight as the reference point again
    & (copy['datetime'] < '2024-05-27 00:05')]
male_midnight = male_midnight.loc[[2104]] # selecting index that is closest to midnight

male_midnight.drop(columns = ['datetime'], inplace = True) # dropping datetime col 
male_midnight = male_midnight[cmpd_names] # only pulling out compounds we have data for
male_midnight = male_midnight.transpose() # want cmpds as row labels
male_midnight.rename(columns = {2104 : 'midnight_ppb'}, inplace = True)
male_midnight = pd.concat([male_midnight, cmpd_k_no3], axis = 1) # combining

cmpd_k_no3 = vp_react_emission_concat[['k_no3_298K']].copy() # rinse and repeat
copy = female_ts.copy()
female_midnight = copy.loc[(copy['datetime'] >= '2024-05-25 23:55')
    & (copy['datetime'] < '2024-05-26 00:05')]
female_midnight = female_midnight.loc[[2075]] 
female_midnight.drop(columns = ['datetime'], inplace = True)
female_midnight = female_midnight[cmpd_names]
female_midnight = female_midnight.transpose() 
female_midnight.rename(columns = {2075 : 'midnight_ppb'}, inplace = True)
female_midnight = pd.concat([female_midnight, cmpd_k_no3], axis = 1)

# calculating exposures for female VOCs at midnight
female_exposures = female_midnight_reactivies[['compounds', 'proposed_cmpd_name', 'female_num_den_midnight', 'no3_reactivity_1/s']].copy() 
female_exposures['pristine_loss_ppb_3hour'] = female_exposures['no3_reactivity_1/s'] * pristine_no3_conc_ppb * (exposure_time_sec * 3) 
female_exposures['polluted_loss_ppb_3hour'] = female_exposures['no3_reactivity_1/s'] * polluted_no3_conc_ppb * (exposure_time_sec * 3) 
female_exposures.set_index('compounds', inplace = True)
female_exposures = pd.concat([female_exposures, female_midnight], axis = 1) # adding in original concentrations
female_exposures.reset_index(inplace = True)
female_exposures.drop([0, 5], inplace = True) # dropping DMTS b/c NaN and dimethyl sulfone b/c data is negative
female_exposures['pristine_remaining_ppb'] = female_exposures['midnight_ppb'] - female_exposures['pristine_loss_ppb_3hour']
female_exposures['polluted_remaining_ppb'] = female_exposures['midnight_ppb'] - female_exposures['polluted_loss_ppb_3hour']
# can't have negative values for the pie charts, so replacing all negatives with zero
female_exposures['pristine_remaining_ppb'] = np.where(female_exposures['pristine_remaining_ppb'] < 0, 0, female_exposures['pristine_remaining_ppb'])
female_exposures['polluted_remaining_ppb'] = np.where(female_exposures['polluted_remaining_ppb'] < 0, 0, female_exposures['polluted_remaining_ppb'])
female_exposures.sort_values(by = ['k_no3_298K'], inplace = True, ascending = False) # sorting by highest reactivity

# rinse and repeat
male_exposures = male_midnight_reactivies[['compounds', 'proposed_cmpd_name', 'male_num_den_midnight', 'no3_reactivity_1/s']].copy()
male_exposures['pristine_loss_ppb_3hour'] = male_exposures['no3_reactivity_1/s'] * pristine_no3_conc_ppb * (exposure_time_sec * 3) # 3 hours
male_exposures['polluted_loss_ppb_3hour'] = male_exposures['no3_reactivity_1/s'] * polluted_no3_conc_ppb * (exposure_time_sec * 3)
male_exposures.set_index('compounds', inplace = True)
male_exposures = pd.concat([male_exposures, male_midnight], axis = 1) 
male_exposures.reset_index(inplace = True)
male_exposures.drop([0, 5], inplace = True) 
male_exposures['pristine_remaining_ppb'] = male_exposures['midnight_ppb'] - male_exposures['pristine_loss_ppb_3hour']
male_exposures['polluted_remaining_ppb'] = male_exposures['midnight_ppb'] - male_exposures['polluted_loss_ppb_3hour']
male_exposures['pristine_remaining_ppb'] = np.where(male_exposures['pristine_remaining_ppb'] < 0, 0, male_exposures['pristine_remaining_ppb'])
male_exposures['polluted_remaining_ppb'] = np.where(male_exposures['polluted_remaining_ppb'] < 0, 0, male_exposures['polluted_remaining_ppb'])
male_exposures.sort_values(by = ['k_no3_298K'], inplace = True, ascending = False)

# setting variables to change add secondary pie charts that are sized by emissions

base_radius = 1 # sizing scale factor for area of plots

female_radius_scale_factor_pristine = mat.sqrt(((female_exposures['pristine_remaining_ppb'].sum())/female_exposures['midnight_ppb'].sum()) / mat.pi) # turning area into radius for sizing
female_radius_scale_factor_polluted = mat.sqrt((female_exposures['polluted_remaining_ppb'].sum()/female_exposures['midnight_ppb'].sum()) / mat.pi)

male_radius_scale_factor_pristine = mat.sqrt(((male_exposures['pristine_remaining_ppb'].sum())/male_exposures['midnight_ppb'].sum()) / mat.pi) # turning area into radius for sizing
male_radius_scale_factor_polluted = mat.sqrt((male_exposures['polluted_remaining_ppb'].sum()/male_exposures['midnight_ppb'].sum()) / mat.pi)



### Figures

#### Main Text Figures

In [ ]:
# bar charts

plt.rcParams["mathtext.default"] = 'regular' # getting rid of italics with subscripts
plt.rcParams["mathtext.fontset"] = 'dejavusans' # setting font for subscripted numbers

# avoiding the extreme ends of the colorbar to get rid of light colors
variable_count = len(percentage.columns)
colormap = plt.get_cmap('twilight')
colors = [colormap(0.1 + (i / (variable_count - 1)) * 0.85) for i in range(variable_count)]

colorbar_width = 0.5 # setting width of bars
totals_width = 0.3

percent_labels = [0, 20, 40, 60, 80, 100] # tick labels
totals_labels = np.linspace(0, 1000, num = 6)
bloom_labels = ['∑ Emitted Compounds [ppb]']
yaxis_labels = np.linspace(0, 27, num = 6)




fig, (ax1, ax4, ax5) = plt.subplots(ncols = 3, 
                               nrows = 1, 
                               dpi = 300,
                               figsize = (7.24, 3.75) # setting to maximum allowed figure width
                              )
plt.subplots_adjust(wspace = 0.4) # adjusting white space between the subplots


ax1 = percentage.plot(kind = 'bar', 
                      stacked = True, 
                      edgecolor = 'black', 
                      linewidth = 0.5, 
                      ax = ax1,
                      width = colorbar_width, 
                      position = 0, 
                      color = colors) # give each cmpd a different  color
ax1.set_ylabel('Fraction ∑ Emitted Compounds (%)', 
               fontsize = 10,
              labelpad = 4)
ax1.set_xlim([-0.1, 1.9])
ax1.set_yticks(ticks = percent_labels, 
               labels = percent_labels, 
               fontsize = 8) # have to call label to be able to control fontsize

ax1.set_ylim([0, 100])

ax2 = ax1.twinx()
ax2 = totals_ppm.plot(kind = 'bar', 
                      color = 'darkgray', 
                      width = totals_width, 
                      edgecolor = 'black', 
                      linewidth = 0.5,
                      position = -1.8) # shifts bar to the right 
ax2.set_ylabel('∑ Emitted Compounds (ppm)', 
               fontsize = 10, 
               labelpad = 10,
              rotation = 270) # pad = adding some space b/ween axis and axis label
ax2.set_ylim([0, 1000])
ax2.set_yticks(ticks = totals_labels,
               labels = totals_labels,
               fontsize = 8)
ax2.yaxis.set_major_formatter(FormatStrFormatter('%g')) # %g removes trailing zeros 
ax2.axhline(y = totals_ppm['total_compound_amt_ppb_female'], # drawing dashed line on top of the gray bars to draw the eye
            xmin = 0.25, # 0 = far left of plot
            xmax = 1, # 1 = far right of plot
            linestyle = 'dotted', 
            color = 'gray',
            alpha = 0.8)
ax2.axhline(y = totals_ppm['total_compound_amt_ppb_male'], # drawing dashed line on top of the gray bars to draw the eye
            xmin = 0.75, # 0 = far left of plot
            xmax = 1, # 1 = far right of plot
            linestyle = 'dotted', 
            color = 'gray',
            alpha = 0.8)


ax1.legend().remove() # hiding the legend to combine with other subplot
ax2.legend().remove() # hiding the legend to combine with other subplot
ax1.set_xticks([0.5, 1.5], 
               labels = ['Female', 'Male'], 
               rotation = 360, 
               fontsize = 10)
ax1.set_title('A. Total Bloom Compounds', 
              fontsize = 8, 
              fontweight = 'bold',
              pad = 8,
              loc = 'left')


ax3 = ax4.twinx()
ax4.yaxis.set_visible(False)
ax3 = graphing_emiss_rates.plot(kind = 'bar', 
                                stacked = True, 
                                edgecolor = 'black', 
                                linewidth = 0.5, 
                                ax = ax3,
                                color = colors) # give each cmpd a different  color
#ax3.set_ylim([0, 27])
ax3.set_ylabel('Log Emission Rates (g $hour^{-1}$)', 
               fontsize = 10, 
               labelpad = 16,
              rotation = 270)
ax3.set_xticklabels(['Female', 'Male'], 
                    rotation = 360, 
                    fontsize = 8)
#ax3.set_yticks(ticks = yaxis_labels, 
             #  labels = yaxis_labels, 
              # fontsize = 8)
ax3.yaxis.set_major_formatter(FormatStrFormatter('%g')) # %g removes trailing zeros 
ax3.legend(loc = 'upper right',  # setting starting legend position
          bbox_to_anchor = (2.55, 1.02), # perfecting x,y coodinates of legend
          ncol = 2, # num cols in legend
          frameon = True,  # adding legend frame
          fontsize = '7', 
          labels = formatted_cmpd_names, # using formatted names in legend labels
          labelspacing = 0.2,
          columnspacing = 0.2 # adjusting spacing of labels and columns in the legend
         )  
ax3.set_title('B. Compound Emission Rates', 
             fontsize = 9,
             fontweight = 'bold', 
             pad = 8,
             loc = 'left')
ax3.set_yscale('log')

ax5.axis('off')

save_path = 'C:\\Users\\rossellr\\OneDrive - Colostate\\Desktop\\CSU\\Research\\Research Projects\\Plants\\CORPSE\\Python\\Final_Figs'
fig_name = today + '_bar_chart_log' + '.svg' # final images need to be .svg

fig_save_path = save_path + "/" + fig_name
#plt.savefig(fname = fig_save_path, bbox_inches = 'tight') #keeps legend included when outputting figure

plt.show()
plt.close()

In [ ]:
# exposure plots

# creating the plot
colors = plt.cm.tab20_r.colors # pulling colors from a qualitative color map

fig, ((ax1, ax2, ax3), (ax4, ax5, ax6)) = plt.subplots(nrows = 2, 
                                                       ncols = 3, 
                                                       dpi = 300,
                                                      figsize = (6, 6))
fig.patch.set_facecolor('#ADCDED')
# setting what I wanted plotted for the female plots
f_params = [
    {'data' : female_exposures['midnight_ppb'],
     'title' : ' '}, 
    {'data' : female_exposures['pristine_remaining_ppb'],
     'title' : 'Pristine Conditions\n(2 ppt NO$_{3}$)'},
    {'data' : female_exposures['polluted_remaining_ppb'],
    'title' : 'Polluted Conditions\n(15 ppt NO$_{3}$)'}
]
# now the male variables
m_params = [
    {'data' : male_exposures['midnight_ppb'],
    # 'title' : 'Midnight Concentrations'
    }, 
    {'data' : male_exposures['pristine_remaining_ppb'],
   #  'title' : 'After 1 Hour'
    },
    {'data' : male_exposures['polluted_remaining_ppb'],
     #'title' : 'After 3 Hours'
    }
]
 # creating the same female pie charts for the different data, pulling from f_params above
for ax, f_params in zip([ax1, ax2, ax3], f_params):
    ax.pie(f_params['data'], 
           colors = colors,
           autopct = lambda pct: '{:1.0f}%'.format(pct) if pct > 4 else '', # setting min for which slices will get % labels 
           textprops = {'fontsize' : 8},
           pctdistance = 1.19,
           rotatelabels = True,
           wedgeprops = {'edgecolor' : 'white', 'linewidth' : 0.5},
           shadow = {'edgecolor' : 'black', 'ox' : 0, 'oy' : 0, 'linewidth' : 3}, # faking an edge outline,
           radius = base_radius
          )
    ax.set_title(f_params['title'],
                fontsize = 11)

for ax, m_params in zip([ax4, ax5, ax6], m_params):
    ax.pie(m_params['data'], 
          colors = colors,
           autopct = lambda pct: '{:1.0f}%'.format(pct) if pct > 4 else '',
           textprops = {'fontsize' : 8},
           pctdistance = 1.19,
           rotatelabels = True,
           wedgeprops = {'edgecolor' : 'white', 'linewidth' : 0.5},
           shadow = {'edgecolor' : 'black', 'ox' : 0, 'oy' : 0, 'linewidth' : 3}, # faking an edge outline,
           radius = base_radius 
          )
             
ax1.text(x = -2.3,  y = 2,  # setting independent plot title\
         s =  'NO$_{3}$ Exposure After 3 Hours',
         fontsize = 14)
ax1.text(x = -2.4,  y = 0.0, 
         s =  'Female',
         fontsize = 12)
ax4.text(x = -2.1,  y = 0.0, 
         s =  'Male',
         fontsize = 12)
ax1.text(x = -0.5,  y = 1.45, 
         s =  'Midnight',
         fontsize = 12)

legend = ax3.legend( # one legend for all
    loc = 'right', 
    bbox_to_anchor = (2.15, 0.2),
    fontsize = 8,
    labels = female_exposures['proposed_cmpd_name'],
    title = 'Increasing NO3 Reactivity',
    title_fontsize = 9,
    alignment = 'right',
    frameon = False,
  #  frameon = True,
  #  framealpha = 1,
   # facecolor = 'white'
    )

# setting the legend title to vertical alignment
legend._legend_box.align = "left"
legend.get_title().set_position((-40, -615))
legend.get_title().set_rotation(90)


plt.subplots_adjust(wspace = 0.01, hspace = 0.0000000000000000000000000000000000000000000000000000000000000000000000000000000000001)

# adding smaller, secondary pie charts that are sized by total emissions
fake_data = [100]
scaling_factors = [
    {'radius' : female_radius_scale_factor_pristine * base_radius},
    {'radius' : female_radius_scale_factor_polluted * base_radius},
    {'radius' : male_radius_scale_factor_pristine * base_radius},
    {'radius' : male_radius_scale_factor_polluted * base_radius}
]
for ax, scaling_factors in zip([ax2, ax3, ax5, ax6], scaling_factors):
    ax.pie(fake_data,
           shadow = {'edgecolor' : 'black', 'ox' : 0, 'oy' : 0, 'linewidth' : 1, 'linestyle' : 'dashed'},
           radius =  scaling_factors['radius'],
           wedgeprops = {'fill' : None, 'linestyle' : 'dashed', 'edgecolor' : 'gray'}
          )
# adding totals underneath each  pie chart
texts = [
    {'text' : f"{female_exposures['midnight_ppb'].sum():0.2f}" + ' ppb'},
    {'text' : f"{female_exposures['pristine_remaining_ppb'].sum():0.2f}" + ' ppb'},
    {'text' : f"{female_exposures['polluted_remaining_ppb'].sum():0.2f}" + ' ppb'},
    {'text' : f"{male_exposures['midnight_ppb'].sum():0.2f}" + ' ppb'},
    {'text' : f"{male_exposures['pristine_remaining_ppb'].sum():0.2f}" + ' ppb'},
    {'text' : f"{male_exposures['polluted_remaining_ppb'].sum():0.2f}" + ' ppb'}
]
for ax, texts in zip([ax1, ax2, ax3, ax4, ax5, ax6], texts):
    ax.text(x = -0.45, y = -1.5,
            s = texts['text'],
            fontsize = 10
            )

for ax in [ax1, ax4]:
    ax.text(x = -1.4, y = -1.5,
            s = 'Total', 
            fontsize = 11)



save_path = 'C:\\Users\\rossellr\\OneDrive - Colostate\\Desktop\\CSU\\Research\\Research Projects\\Plants\\CORPSE\\Python\\Final_Figs'
fig_name = today + '_no3_exposures' + '.svg'

fig_save_path = save_path + "/" + fig_name
#plt.savefig(fname = fig_save_path, bbox_inches = 'tight') #keeps legend included when outputting figure

plt.show()
plt.close()


#### SI Figures

In [ ]:
# bar chart, linear scale

plt.rcParams["mathtext.default"] = 'regular' # getting rid of italics with subscripts
plt.rcParams["mathtext.fontset"] = 'dejavusans' # setting font for subscripted numbers

# avoiding the extreme ends of the colorbar to get rid of light colors
variable_count = len(percentage.columns)
colormap = plt.get_cmap('twilight')
colors = colors = [colormap(0.1 + (i / (variable_count - 1)) * 0.85) for i in range(variable_count)]

colorbar_width = 0.5 # setting width of bars
totals_width = 0.3

percent_labels = [0, 20, 40, 60, 80, 100] # tick labels
totals_labels = np.linspace(0, 1000, num = 6)
bloom_labels = ['∑ Emitted Compounds [ppb]']
yaxis_labels = np.linspace(0, 27, num = 6)




fig, (ax1, ax4, ax5) = plt.subplots(ncols = 3, 
                               nrows = 1, 
                               dpi = 300,
                               figsize = (7.24, 3.75) # setting to maximum allowed figure width
                              )
plt.subplots_adjust(wspace = 0.4) # adjusting white space between the subplots


ax1 = percentage.plot(kind = 'bar', 
                      stacked = True, 
                      edgecolor = 'black', 
                      linewidth = 0.5, 
                      ax = ax1,
                      width = colorbar_width, 
                      position = 0, 
                      color = colors) # give each cmpd a different  color
ax1.set_ylabel('Fraction ∑ Emitted Compounds (%)', 
               fontsize = 10,
              labelpad = 4)
ax1.set_xlim([-0.1, 1.9])
ax1.set_yticks(ticks = percent_labels, 
               labels = percent_labels, 
               fontsize = 8) # have to call label to be able to control fontsize

ax1.set_ylim([0, 100])

ax2 = ax1.twinx()
ax2 = totals_ppm.plot(kind = 'bar', 
                      color = 'darkgray', 
                      width = totals_width, 
                      edgecolor = 'black', 
                      linewidth = 0.5,
                      position = -1.8) # shifts bar to the right 
ax2.set_ylabel('∑ Emitted Compounds (ppm)', 
               fontsize = 10, 
               labelpad = 10,
              rotation = 270) # pad = adding some space b/ween axis and axis label
ax2.set_ylim([0, 1000])
ax2.set_yticks(ticks = totals_labels,
               labels = totals_labels,
               fontsize = 8)
ax2.yaxis.set_major_formatter(FormatStrFormatter('%g')) # %g removes trailing zeros 
ax2.axhline(y = totals_ppm['total_compound_amt_ppb_female'], # drawing dashed line on top of the gray bars to draw the eye
            xmin = 0.25, # 0 = far left of plot
            xmax = 1, # 1 = far right of plot
            linestyle = 'dotted', 
            color = 'gray',
            alpha = 0.8)
ax2.axhline(y = totals_ppm['total_compound_amt_ppb_male'], # drawing dashed line on top of the gray bars to draw the eye
            xmin = 0.75, # 0 = far left of plot
            xmax = 1, # 1 = far right of plot
            linestyle = 'dotted', 
            color = 'gray',
            alpha = 0.8)


ax1.legend().remove() # hiding the legend to combine with other subplot
ax2.legend().remove() # hiding the legend to combine with other subplot
ax1.set_xticks([0.5, 1.5], 
               labels = ['Female', 'Male'], 
               rotation = 360, 
               fontsize = 10)
ax1.set_title('A. Total Bloom Compounds', 
              fontsize = 8, 
              fontweight = 'bold',
              pad = 8,
              loc = 'left')

g_emiss = graphing_emiss_rates #/ 1000
ax3 = ax4.twinx()
ax4.yaxis.set_visible(False)
ax3 = g_emiss.plot(kind = 'bar', 
                                stacked = True, 
                                edgecolor = 'black', 
                                linewidth = 0.5, 
                                ax = ax3,
                                color = colors) # give each cmpd a different  color
#ax3.set_ylim([0, 27])
ax3.set_ylabel('Emission Rates (g $hour^{-1}$)', 
               fontsize = 10, 
               labelpad = 16,
              rotation = 270)
ax3.set_xticklabels(['Female', 'Male'], 
                    rotation = 360, 
                    fontsize = 8)
#ax3.set_yticks(ticks = yaxis_labels, 
             #  labels = yaxis_labels, 
              # fontsize = 8)
ax3.yaxis.set_major_formatter(FormatStrFormatter('%g')) # %g removes trailing zeros 
ax3.legend(loc = 'upper right',  # setting starting legend position
          bbox_to_anchor = (2.55, 1.02), # perfecting x,y coodinates of legend
          ncol = 2, # num cols in legend
          frameon = True,  # adding legend frame
          fontsize = '7', 
          labels = formatted_cmpd_names, # using formatted names in legend labels
          labelspacing = 0.2,
          columnspacing = 0.2 # adjusting spacing of labels and columns in the legend
         )  
ax3.set_title('B. Compound Emission Rates', 
             fontsize = 9,
             fontweight = 'bold', 
             pad = 8,
             loc = 'left')
#ax3.set_yscale('log')

ax5.axis('off')

save_path = ''# enter save path here
fig_name = today + '_bar_chart_linear' + '.svg' # final images need to be .svg

fig_save_path = save_path + "/" + fig_name
#plt.savefig(fname = fig_save_path, bbox_inches = 'tight') #keeps legend included when outputting figure

plt.show()
plt.close()

In [ ]:
# oxidation plots for SI:
# NO3 oxidation

no_labels = ['', '', ''] # removing labels from the individual wedges for clarity


# Total NO3 reactivities for male and female blooms at midnight
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (10, 12), dpi = 300) # 300 dpi is required for most publications


data = [
    {'no3_react' : top_female_no3_reactivies['no3_reactivity_percent'],
    'color_key' : ['#51536F', '#F6BE9A', '#E5446D']},
    {'no3_react' : top_male_no3_reactivies['no3_reactivity_percent'],
    'color_key' : ['#E1CACA', '#44A1A0', '#E5446D']}
]
    
for ax, data in zip([ax1, ax2], data):
    ax.pie(data['no3_react'],
        labels = no_labels, # labels will label each pie slice 
        pctdistance = 1.14, # setting distance of the percents from the chart
        autopct = lambda pct: '{:1.1f}%'.format(pct) if pct > 5 else '', # setting limits to what percentages are shown
        colors = data['color_key'],
        hatch = ['', '', '//'], # only want hatch on pink slice to increase visibility of printed images
        textprops = {'fontsize' : 10} 
          )

ax1.legend(loc = 'center',
           bbox_to_anchor = (0.85, 0.3, 0.5, 1.2), # setting distance of legend from plot
           labels = top_female_no3_reactivies['proposed_cmpd_name'],
           fontsize = 12,
          title = 'Female') # settting independent labels for the legend

ax2.legend(loc = 'center', 
           bbox_to_anchor = (-0.35, -0.42, 0.5, 1.2), 
           labels = top_male_no3_reactivies['proposed_cmpd_name'],
          fontsize = 12,
            title = 'Male')


ax1.set_title('NO$_{3}$ Reactivity: ' + f"{top_female_no3_reactivies['no3_reactivity_1/s'].sum():0.2f}"  + ' (s$^-$$^1$)' + '\nMidnight, Female Bloom', 
              fontsize = 14) 

ax2.set_title('NO$_{3}$ Reactivity: ' + f"{top_male_no3_reactivies['no3_reactivity_1/s'].sum():0.4f}" + ' (s$^-$$^1$)' + '\nMidnight, Male Bloom', 
              fontsize = 14)

save_path = '' # enter save path here
fig_name = today + '_FvM_NO3_piecharts' + '.svg'
fig_save_path = save_path + "/" + fig_name
#plt.savefig(fname = fig_save_path, bbox_inches = 'tight') #keeps legend included when outputting figure

plt.show()
plt.close()


In [ ]:
# # Total O3 reactivities for male and female blooms at midnight (not sized by anything)


data = [
    {'o3_react' : top_female_o3_reactivies['o3_reactivity_percent']},
    {'o3_react' : top_male_o3_reactivies['o3_reactivity_percent']}
]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (10, 12), dpi = 300)

for ax, data in zip([ax1, ax2], data):
    ax.pie(data['o3_react'],
           labels = no_labels,
           pctdistance = 1.2,
        autopct = lambda pct: '{:1.1f}%'.format(pct) if pct > 3 else '', 
        colors = ['#A7A7A9', '#251351', '#E5446D'], 
        textprops = {'fontsize' : 14},
        hatch = ['', '', '//']
          )


ax1.legend(loc = 'center', 
           bbox_to_anchor = (0.82, 0.3, 0.5, 1.2), 
           labels = top_male_o3_reactivies['proposed_cmpd_name'], 
           fontsize = 12)
#ax2.legend(loc = 'center', 
           #bbox_to_anchor = (0.82, 0.3, 0.5, 1.2), 
          # labels = top_female_o3_reactivies['proposed_cmpd_name'], 
           #fontsize = 12)
ax1.set_title('O$_{3}$ Reactivity: ' + f"{top_female_o3_reactivies['o3_reactivity_1/s'].sum():0.2e}" + ' (s$^-$$^1$)' + '\nMidnight, Female Bloom' , 
              fontsize = 14) 

ax2.set_title('O$_{3}$ Reactivity: ' + f"{top_male_o3_reactivies['o3_reactivity_1/s'].sum():0.2e}" + ' (s$^-$$^1$)' + '\nMidnight, Male Bloom' , 
              fontsize = 14)

save_path = '' # enter save path here
fig_name = today + '_FvM_O3_piecharts' + '.svg'
fig_save_path = save_path + "/" + fig_name
#plt.savefig(fname = fig_save_path, bbox_inches = 'tight') #keeps legend included when outputting figure

plt.show()
plt.close()


In [ ]:
# Total OH reactivities for male and female blooms at midnight (not sized by anything)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (10, 12), dpi = 300)

data = [
    {'oh_react' : top_female_oh_reactivies['oh_reactivity_percent'],
    'colors' : ['#51536F', '#F6BE9A', '#E5446D']},
    {'oh_react' : top_male_oh_reactivies['oh_reactivity_percent'],
    'colors' : ['#44A1A0', '#F6BE9A', '#E5446D']}
]

for ax, data in zip([ax1, ax2], data):
    ax.pie(data['oh_react'], 
           labels = no_labels, 
           pctdistance = 1.2,
           autopct = lambda pct: '{:1.1f}%'.format(pct) if pct > 3 else '', 
           colors = data['colors'], 
           hatch = ['', '', '//'], 
           textprops = {'fontsize' : 12}
          )

ax1.legend(loc = 'center', 
           bbox_to_anchor = (0.8, 0.35, 0.5, 1.2), 
           labels = top_female_oh_reactivies['proposed_cmpd_name'], 
           fontsize = 12,
          title = 'Female')
ax2.legend(loc = 'center', 
           bbox_to_anchor = (0.83, 0.35, 0.5, 1.2), 
           labels = top_male_oh_reactivies['proposed_cmpd_name'], 
           fontsize = 12,
          title = 'Male')

ax1.set_title('OH Reactivity: ' + f"{top_female_oh_reactivies['oh_reactivity_1/s'].sum():0.0f}" + ' (s$^-$$^1$)'  + '\nMidnight, Female Bloom'  , 
              fontsize = 14)
ax2.set_title('OH Reactivity: ' + f"{top_male_oh_reactivies['oh_reactivity_1/s'].sum():0.3f}" + ' (s$^-$$^1$)' + '\nMidnight, Male Bloom' ,
              fontsize = 14)

save_path = '' # enter save path here
fig_name = today + '_FvM_OH_piecharts' + '.svg'
fig_save_path = save_path + "/" + fig_name
#plt.savefig(fname = fig_save_path, bbox_inches = 'tight') #keeps legend included when outputting figure

plt.show()
plt.close()

In [ ]:
# VP vs main S cmpds


xlabels = big_s_cmpds['cmpd_names'] + ' (' + big_s_cmpds['vp_mmHg'].astype(str) + ' mmHg)' # combining VP w/ name for visualization
xticks  = big_s_cmpds['cmpd_names']

width = 0.25
labels = big_s_cmpds.index

br1 = np.arange(len(big_s_cmpds['total_compound_amt_ppb_female']))
br2 = [x + width for x in br1]

fig, ax = plt.subplots(figsize = (6,4), dpi = 300)

plt.bar(br1, big_s_cmpds['total_compound_amt_ppb_female'], width = width, label = 'Female', color = '#B8B8D1')
plt.bar(br2, big_s_cmpds['total_compound_amt_ppb_male'], width = width, label = 'Male', color = '#3D4066')

plt.xticks([r + width for r in range(len(big_s_cmpds['total_compound_amt_ppb_female']))], xlabels, rotation = 30)
plt.yscale('log')
plt.legend(loc = 'upper right')
plt.ylabel('∑ Emitted Compounds [ppb]\nLog Scale')

save_path = '' # enter save path here
fig_name = 'MaleVsFemale_VP_logScale_' + today + '.png'
fig_save_path = save_path + "/" + fig_name
#plt.savefig(fname = fig_save_path, bbox_inches = 'tight') #keeps legend included when outputting figure

plt.show()


In [ ]:
# ptr vs gc data for S cmpds

labels = ['05/26 9:00', '05/26 10:00', '05/26 12:00', '05/26 13:00', '05/26 15:00', 
         '05/26 17:00', '05/27 9:00', '05/27 11:00']
#labels = pd.to_datetime(labels, format = '%Y-%m-%d %H:%M')

fig, ((ax1), (ax2)) = plt.subplots(ncols = 1, nrows = 2, sharex = True,  dpi = 300, figsize = (5, 4))

ax1.plot(gc_samp_copy['Sample_ID'], gc_samp_copy['Dimethyl disulfide'], label = 'DMDS (GC)',
         linestyle = '', linewidth = 0.25, marker = 'v', color = '#938274', markersize = 6, alpha = 0.5)
ax1.set_yticks(np.linspace(0, 0.015, 3)) # setting ylim and setting number of ticks to 3 
ax11 = ax1.twinx() # create another y axis
ax11.plot(PTR_means.index, PTR_means['C2H7S2+'], label = 'DMDS (PTR)',
         linestyle = '', marker = 'o', color = '#938274', markersize = 6)
ax11.set_yticks(np.linspace(0, 0.08, 3))

ax2.plot(gc_samp_copy['Sample_ID'], gc_samp_copy['Dimethyl sulfone'], label = 'Dimethyl sulfone (GC)',
         linestyle = '', linewidth = 0.25, marker = 'v', color = '#451F55', markersize = 6, alpha = 0.5)
ax2.set_yticks(np.linspace(0, 0.006, 3)) # setting ylim and setting number of ticks to 3 
ax12 = ax2.twinx() # create another y axis
ax12.plot(PTR_means.index, PTR_means['C2H7O2S+'], label = 'Dimethyl sulfone (PTR)',
         linestyle = '', marker = 'o', color = '#451F55', markersize = 6)
ax12.set_yticks(np.linspace(0, 0.15, 3))

ax2.xaxis.set_major_formatter(md.DateFormatter('%D-%m'))
ax2.set_xticks(range(len(labels)), labels = labels, rotation = 30)

fig.text(-0.01, 0.53, 'GC Conc (ppb)', va = 'center', rotation = 'vertical', fontsize = 10) # setting new yaxis labels
fig.text(0.975, 0.55, 'PTR Conc (ppb)', va = 'center', rotation = 'vertical', fontsize = 10)

for ax in [ax1, ax2, ax3, ax4, ax5]: # setting shared parameters between all the subplots
    ax.legend(loc = 'upper left', fontsize = 8)

for ax in [ax11, ax12, ax13, ax14, ax15]:
    ax.legend(loc = 'upper right', fontsize = 8)

plt.tight_layout()

save_path = '' # enter save path here
fig_name = today + '_sulfur_gc_ptr_compre' + '.svg'

fig_save_path = save_path + "/" + fig_name
#plt.savefig(fname = fig_save_path, bbox_inches = 'tight') #keeps legend included when outputting figure

plt.show()
plt.close()


In [ ]:
# ptr vs gc comparison: bvocs, splitting up cyclopentane 

labels = ['05/26 9:00', '05/26 10:00', '05/26 12:00', '05/26 13:00', '05/26 15:00', 
         '05/26 17:00', '05/27 9:00', '05/27 11:00']
#labels = pd.to_datetime(labels, format = '%Y-%m-%d %H:%M')

fig, ((ax1), (ax2), (ax3), (ax4), (ax25), (ax5)) = plt.subplots(ncols = 1, nrows = 6, sharex = True,  dpi = 300, figsize = (5, 8))

ax1.plot(gc_samp_copy['Sample_ID'], gc_samp_copy['Hexanal'], label = 'Hexanal (GC)',
         linestyle = '', linewidth = 0.25, marker = 'v', color = 'green', markersize = 6, alpha = 0.5)
ax1.set_yticks(np.linspace(0, 0.2, 3)) # setting ylim and setting number of ticks to 3 
ax11 = ax1.twinx() # create another y axis
ax11.plot(PTR_means.index, PTR_means['C6H13O+'], label = 'Hexanal (PTR)',
         linestyle = '', marker = 'o', color = 'green', markersize = 6)
ax11.set_yticks(np.linspace(0, 0.05, 3))

ax2.plot(gc_samp_copy['Sample_ID'], gc_samp_copy['Benzene'], label = 'Benzene (GC)',
         linestyle = '', linewidth = 0.25, marker = 'v', color = 'darkorange', markersize = 6, alpha = 0.5)
ax2.set_yticks(np.linspace(0, 0.3, 3))
ax12 = ax2.twinx() # create another y axis
ax12.plot(PTR_means.index, PTR_means['C6H7+'], label = 'Benzene (PTR)',
         linestyle = '', marker = 'o', color = 'darkorange', markersize = 6)
ax12.set_yticks(np.linspace(0, 0.3, 3))

ax3.plot(gc_samp_copy['Sample_ID'], gc_samp_copy['Phenol'], label = 'Phenol (GC)',
         linestyle = '', linewidth = 0.25, marker = 'v', color = 'gray', markersize = 6, alpha = 0.5)
ax3.set_yticks(np.linspace(0, 0.5, 3))
ax13 = ax3.twinx() # create another y axis
ax13.plot(PTR_means.index, PTR_means['C6H7O+'], label = 'Phenol (PTR)',
         linestyle = '', marker = 'o', color = 'gray', markersize = 6)
ax13.set_yticks(np.linspace(0, 0.5, 3))

ax4.plot(gc_samp_copy['Sample_ID'], gc_samp_copy['Benzaldehyde'], label = 'Benzaldehyde (GC)',
         linestyle = '', linewidth = 0.25, marker = 'v', color = 'darkblue', markersize = 6, alpha = 0.5)
ax4.set_yticks(np.linspace(0, 0.5, 3))
ax14 = ax4.twinx() # create another y axis
ax14.plot(PTR_means.index, PTR_means['C7H7O+'], label = 'Benzaldehyde (PTR)',
         linestyle = '', marker = 'o', color = 'darkblue', markersize = 6)
ax14.set_yticks(np.linspace(0, 0.1, 3))

# top graph of the broken axis for cyclopentance
ax25.plot(gc_samp_copy['Sample_ID'], gc_samp_copy['Cyclopentane'], label = 'Cyclopentane (GC)', alpha = 0.5,
        linestyle = '', linewidth = 0.25, marker = 'v', color = 'red', markersize = 6)
ax25.set_ylim([4, 4.5])
ax25.set_yticks(np.linspace(4.25, 4.5, 2))
ax25.xaxis.set_visible(False)
ax25.spines.bottom.set_visible(False)
#ax25.spines.right.set_visible(False)

# bottom graph of the broken axis for cyclopentance
ax5.plot(gc_samp_copy['Sample_ID'], gc_samp_copy['Cyclopentane'], label = 'Cyclopentane (GC)', alpha = 0.5,
        linestyle = '', linewidth = 0.25, marker = 'v', color = 'red', markersize = 6)
ax5.set_ylim([0, 0.5])
ax5.set_yticks(np.linspace(0, 0.25, 2))
ax5.spines.top.set_visible(False)
# setting those horizonantal lines
# source code for the split axis: https://matplotlib.org/stable/gallery/subplots_axes_and_figures/broken_axis.html
d = 0.5  # proportion of vertical to horizontal extent of the slanted line
kwargs = dict(marker=[(-1, -d), (1, d)], markersize = 12,
              linestyle = "none", color = 'k', mec = 'k', mew = 1, clip_on = False)
ax25.plot([0, 1], [0, 0], transform = ax25.transAxes, **kwargs)
ax5.plot([0, 1], [1, 1], transform = ax5.transAxes, **kwargs)


ax15 = ax5.twinx() # create another y axis
ax15.plot(PTR_means.index, PTR_means['C5H11+'], label = 'Cyclopentane (PTR)',
        linestyle = '', marker = 'o', color = 'red', markersize = 6)
ax15.set_ylim([0, 0.2])
ax15.set_yticks(np.linspace(0, 0.1, 2))
ax15.spines.top.set_visible(False)


ax5.xaxis.set_major_formatter(md.DateFormatter('%D-%m'))
ax5.set_xticks(range(len(labels)), labels = labels, rotation = 30)

fig.text(-0.01, 0.53, 'GC Conc (ppb)', va = 'center', rotation = 'vertical', fontsize = 10) # setting new yaxis labels
fig.text(0.975, 0.55, 'PTR Conc (ppb)', va = 'center', rotation = 'vertical', fontsize = 10)

for ax in [ax1, ax2, ax3, ax4, ax5]: # setting shared parameters between all the subplots
    ax.legend(loc = 'upper left', fontsize = 8)

for ax in [ax11, ax12, ax13, ax14, ax15]:
    ax.legend(loc = 'upper right', fontsize = 8)

plt.tight_layout()

save_path = 'C:\\Users\\rossellr\\OneDrive - Colostate\\Desktop\\CSU\\Research\\Research Projects\\Plants\\CORPSE\\Python\\Final_Figs'
fig_name = today + '_bvocs_gc_ptr_compre' + '.svg'

fig_save_path = save_path + "/" + fig_name
#plt.savefig(fname = fig_save_path, bbox_inches = 'tight') #keeps legend included when outputting figure

plt.show()
plt.close()
